In [100]:
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
import pandas as pd
import librosa
from pydub import AudioSegment
import math
import random
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import csv
import shutil

In [101]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 1000
        t2 = to_min  * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export('./splitMusic' + '/' + split_filename, format="wav")
        
    def multiple_split(self, sec_per_split, start_sec, end_bef, rand_amount):
        total_mins = math.floor(self.get_duration())
        #print(total_mins)
        for i in range(rand_amount):
            rand_start = random.randint(start_sec, total_mins-end_bef)
            split_fn = self.filename.replace('.wav','') + '_' + str(rand_start) +'.wav'
            self.single_split(rand_start, rand_start+sec_per_split, split_fn)
            print(str(i) + ' Done')
            #if i == total_mins - min_per_split:
                #print('All splited successfully')

In [121]:
def plot_spectrogram(Y, sr, hop_length, filename, y_axis="log"):
    fig = plt.figure(figsize=(25, 10))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    librosa.display.specshow(Y, sr=sr, hop_length=hop_length, y_axis=y_axis)
    ax.axes.yaxis.set_visible(False)
    fig.savefig("./images/"+filename+".png", bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [30]:
def createCSV():
    header = ['name', 'bpm']
    lst = os.listdir("./data/images")
    #data = convert(header, 0, lst)
    #print(data)
    with open('data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        totLst = []
        for i in range (len(lst)):
            tmpLst = []
            tmpLst.append(lst[i][4:])
            tmpLst.append(lst[i][0:3]) 
            totLst.append(tmpLst)
            print(tmpLst)
            writer.writerows([tmpLst])


In [128]:
def sortImgs():
    path = '.'
    lst = os.listdir("./images")
    if not os.path.exists(path+"/data"):
        os.makedirs(path+'/data')
    for fle in lst:
        bpm = fle[0:3]
        genre = fle[4:7]
        if os.path.exists(path+"/data/"+bpm):
            shutil.copy(path+'/images/'+fle, path+'/data/'+bpm+'/'+fle)
        else:
            os.makedirs(path+'/data/'+bpm)
            shutil.copy(path+'/images/'+fle, path+'/data/'+bpm+'/'+fle)
            
            
        if os.path.exists(path+"/data/"+genre):
            shutil.copy(path+'/images/'+fle, path+'/data/'+genre+'/'+fle)
        else:
            os.makedirs(path+'/data/'+genre)
            shutil.copy(path+'/images/'+fle, path+'/data/'+genre+'/'+fle)

In [17]:
path = "./a/reg/130"
lst = os.listdir(path)
for fle in lst:
    bpm = path[8:11] + "_"
    genre = path[4:7] + "_"
    print(bpm)
    print(genre)
    shutil.copy(path+"/"+fle, "./music/"+bpm+genre+fle)
        

130_
reg_
130_
reg_
130_
reg_
130_
reg_
130_
reg_
130_
reg_


In [130]:
path, dirs, files = next(os.walk("./splitMusic"))
file_count = len(files)
print(file_count)
path, dirs, files = next(os.walk("./images"))
file_count = len(files)
print(file_count)

3013
3013


In [114]:
#function to splice a wav file into random multiple segments
mylist = os.listdir("./music")
#print(mylist)
#print (len(mylist))
FRAME_SIZE = 2048
HOP_SIZE = 512

for x in mylist:
    print(x)
    folder = './music'
    file = x
    split_wav = SplitWavAudioMubin(folder, file)
    split_wav.multiple_split(sec_per_split=4, start_sec=45,  end_bef=30, rand_amount=5)

120_edm_badIdeas.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_edm_cocaineModel.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_edm_Crystal Castles - Vanished.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_edm_dancin.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_edm_Imagine.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_edm_loveRunsOut.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_Caroline - Aminé.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_Jaden Smith- Icon.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_NF - The Search (lyrics).wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_NLE Choppa - Shotta Flow (Lyrics).wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_Tierra Whack – Peppers And Onions.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rap_YK OSIRIS -  'VALENTINE'.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rck_Bouncing Souls - Home.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rck_Car Seat Headrest - It's Only Sex.wav
0 Done
1 Done
2 Done
3 Done
4 Done
120_rck_Punk Junkies From New York.wav
0 Done
1 Done
2 Do

In [127]:
mylist = os.listdir("./splitMusic")
FRAME_SIZE = 2048
HOP_SIZE = 512
for x in mylist:
    if(((x[0:3] == "130")) & ((x[4:7] == "reg"))):
        filename = x.replace('.wav','')
        filelocation = "./splitMusic/"+ x
        scale, sr = librosa.load(filelocation)
        S_scale = librosa.stft(scale, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
        Y_scale = np.abs(S_scale) ** 2
        Y_scale.shape
        Y_log_scale = librosa.power_to_db(Y_scale)
        plot_spectrogram(Y_log_scale, sr, HOP_SIZE, filename, y_axis="log")


In [ ]:
createCSV()

In [131]:
sortImgs()

In [73]:
mylist = os.listdir("./splitMusic")
FRAME_SIZE = 2048
HOP_SIZE = 512
for x in mylist:
    if((not(x[0:3] == "120")) & (not(x[4:7] == "edm"))):
        continue
    print(x)

120_edm_badIdeas_105.wav
120_edm_badIdeas_106.wav
120_edm_badIdeas_107.wav
120_edm_badIdeas_111.wav
120_edm_badIdeas_115.wav
120_edm_badIdeas_117.wav
120_edm_badIdeas_118.wav
120_edm_badIdeas_121.wav
120_edm_badIdeas_123.wav
120_edm_badIdeas_125.wav
120_edm_badIdeas_127.wav
120_edm_badIdeas_133.wav
120_edm_badIdeas_134.wav
120_edm_badIdeas_47.wav
120_edm_badIdeas_48.wav
120_edm_badIdeas_50.wav
120_edm_badIdeas_52.wav
120_edm_badIdeas_57.wav
120_edm_badIdeas_59.wav
120_edm_badIdeas_60.wav
120_edm_badIdeas_63.wav
120_edm_badIdeas_67.wav
120_edm_badIdeas_69.wav
120_edm_badIdeas_70.wav
120_edm_badIdeas_71.wav
120_edm_badIdeas_76.wav
120_edm_badIdeas_78.wav
120_edm_badIdeas_79.wav
120_edm_badIdeas_80.wav
120_edm_badIdeas_81.wav
120_edm_badIdeas_83.wav
120_edm_badIdeas_86.wav
120_edm_badIdeas_88.wav
120_edm_badIdeas_92.wav
120_edm_badIdeas_93.wav
120_edm_badIdeas_96.wav
120_edm_badIdeas_97.wav
120_edm_badIdeas_98.wav
120_edm_cocaineModel_102.wav
120_edm_cocaineModel_106.wav
120_edm_cocaineMo